如何计算一个离散时间、有限状态的马尔科夫过程：$$\Pi =(\pi_1,...,\pi_n)$$服从$$\sum_i^n\pi_i=1$$且$$\Pi_{t+1} =\Pi_tP$$其中$P=(p_{ij}),p_{ij} = \mathbb{P}(X_{t+1}=j|X_t =i)$

那么如何求解这个马尔可夫过程$\Pi$的稳态分布呢？我们可以考虑如下线性方程组的解：

$$
\begin{cases}
(P^T - I)\Pi^T = 0 \\
\mathbf{1}^T \Pi^T = \mathbf{1}
\end{cases}
$$

第一种方法是利用矩阵求逆直接求解上述线性方组，但可能遭遇数值不稳定的问题！

第二种方法是从下述公式出发$$P^T\Pi^T = 1\times\Pi^T$$
显然$(1,\Pi^T)$可以看作是$P^T$的特征值和特征向量，那么我们只有求解$P^T$的特征向量，找出与特征值为$1$所对应的特征向量，再将其各分量全部标准化即可。

第三种办法则是简单粗暴的模拟方法，即从给定的概率向量开始，依照状态转移方程迭代计算，直到误差不超过容忍度：
$$||\Pi^{(0)}||=1,\ \Pi^{(1)}=\Pi^{(0)}P$$

In [18]:
using LinearAlgebra

"""
distributionSS(P::AbstractMatrix{<:Real}; method=:eigen, maxit = 1e5)
基于特征向量法和模拟方法计算状态转移矩阵 P 的稳态分布 π，
要求 P 是行和为 1 的方阵。
"""
function distributionSS(P::AbstractMatrix{<:Real}; method=:eigen,maxit = 1e5)
    P = transpose(P)  #先转置为方便求解的矩阵形式
    n, m = size(P)
    @assert n == m "P 必须是方阵"
    # 检验转置后的矩阵列和为 1，也就是说原始状态转移矩阵的行和为 1
    tol = 1e-12
    @assert all(abs.(sum(P, dims=1)[:] .- 1) .< tol) "P 的每行和必须为 1"
    if method == :eigen
        # 对 P 求右特征分解
        F = eigen(P)
        # 定位特征值最接近 1 的索引
        idx = argmin(abs.(F.values .- 1))
        # 提取对应的特征向量，并取实部
        v = real(F.vectors[:, idx])
        # 归一化特征向量
        π = v / sum(v)
        return π
    elseif method == :simulate
        v = zeros(size(P, 1))
        v[1] = 1.0;
        for it in 1:maxit
            v = P * v
           if it % 100 ==0 && all(abs.(v-P*v).<1e-12)
                break
            end
        end
        return v

    else
        error("不支持的方法：$method")
    end
end

# 使用示例
P = [0.9 0.1 0.0;
     0.1 0.5 0.4;
     0.0 0.2 0.8]  # 列和均为 1
π = distributionSS(P)
println("稳态分布 π = ", π)


稳态分布 π = [0.2500000000000007, 0.24999999999999983, 0.4999999999999995]
